Quick dynaprog test..

In [15]:
"""
Uses the sequential dynamic programming function to find a proportional allocation
of items to agents with different valuations, with a largest sum of utilities (utilitarian value).
The input is a valuation-matrix v, where v[i][j] is the value of agent i to item j.
The states are of the form  (v1, v2, ..., vn) where n is the number of agents.
The "vi" are the value of bundle i to agent i.
Programmer: Erel Segal-Halevi
Since: 2021-12
"""

import dynprog
from dynprog.sequential import SequentialDynamicProgram

import math, logging
from typing import *

logger = logging.getLogger(__name__)


####
## COMMON BLOCK
####

def items_as_value_vectors(valuation_matrix):
    """
    Convert a valuation matrix (an input to a fair division algorithm) into a list of value-vectors.
    Each value-vector v represents an item: v[i] is the value of the item for agent i (i = 0,...,n-1).
    The last element, v[n], is the item index.
    >>> items_as_value_vectors([[11,22,33],[44,55,66]])
    [[11, 44, 0], [22, 55, 1], [33, 66, 2]]
    """
    num_of_agents   = len(valuation_matrix)
    num_of_items    = len(valuation_matrix[0])
    return [  # Each item is represented by a vector of values - a value for each agent. The last value is the item index.
        [valuation_matrix[agent_index][item_index] for agent_index in range(num_of_agents)] + [item_index]
        for item_index in range(num_of_items)
    ]



def add_input_to_bin_sum(bin_sums:list, bin_index:int, input:int):
    """
    Adds the given input integer to bin #bin_index in the given list of bins.
    >>> add_input_to_bin_sum([11, 22, 33], 0, 77)
    (88, 22, 33)
    >>> add_input_to_bin_sum([11, 22, 33], 1, 77)
    (11, 99, 33)
    >>> add_input_to_bin_sum([11, 22, 33], 2, 77)
    (11, 22, 110)
    """
    new_bin_sums = list(bin_sums)
    new_bin_sums[bin_index] = new_bin_sums[bin_index] + input
    return tuple(new_bin_sums)


def add_input_to_agent_value(agent_values:list, agent_index:int, item_values:list):
    """
    Update the state of a dynamic program by giving an item to a specific agent.
    :param agent_values: the current vector of agent values, before adding the new item.
    :param agent_index: the agent to which the item is given.
    :param item_values: a list of values: input[i] represents the value of the current item for agent i.
    >>> add_input_to_agent_value([11, 22, 33], 0, [55,66,77,1])
    (66, 22, 33)
    >>> add_input_to_agent_value([11, 22, 33], 1, [55,66,77,1])
    (11, 88, 33)
    >>> add_input_to_agent_value([11, 22, 33], 2, [55,66,77,1])
    (11, 22, 110)
    """
    return add_input_to_bin_sum(agent_values, agent_index, item_values[agent_index])


def add_input_to_bin(bins:list, agent_index:int, item_index:int):
    """
    Update the solution of a dynamic program by giving an item to a specific agent.
    
    :param bins: the current vector of agent bundles, before adding the new item.
    :param agent_index: the agent to which the item is given.
    :param item_index: the index of the given item.
    Adds the given input integer to bin #agent_index in the given list of bins.
    >>> add_input_to_bin([[11,22], [33,44], [55,66]], 1, 1)
    [[11, 22], [33, 44, 1], [55, 66]]
    """
    new_bins = list(bins)
    new_bins[agent_index] = new_bins[agent_index]+[item_index]
    return new_bins

#### 
## UM within EF1 Code
###


def utilitarian_ef1_value(valuation_matrix, efx=False):
    """
    Returns the maximum utilitarian value in a ef1 allocation - does *not* return the partition itself.
    >>> dynprog.logger.setLevel(logging.WARNING)
    >>> utilitarian_ef1_value([[11,0,11],[33,44,55]])
    110.0
    >>> utilitarian_ef1_value([[11,0,11],[33,44,55]],efx=True)
    110.0
    >>> utilitarian_ef1_value([[11,22,33,44],[44,33,22,11]])
    154.0
    >>> utilitarian_ef1_value([[11,22,33,44],[44,33,22,11]],efx=True)
    154.0
    >>> utilitarian_ef1_value([[11,0,11,11],[0,11,11,11],[33,33,33,33]])
    88.0
    >>> utilitarian_ef1_value([[11,0,11,11],[0,11,11,11],[33,33,33,33]],efx=True)
    88.0
    >>> utilitarian_ef1_value([[11],[22]])
    22.0
    >>> utilitarian_ef1_value([[11],[22]],efx=True)
    22.0
    >>> utilitarian_ef1_value([[98,91,29,50,76,94],[43,67,93,35,49,12],[45,10,62,47,82,60]])
    505.0
    >>> utilitarian_ef1_value([[98,91,29,50,76,94],[43,67,93,35,49,12],[45,10,62,47,82,60]],efx=True)
    481.0
    """
    items = items_as_value_vectors(valuation_matrix)
    return EF1PartitionDP(valuation_matrix, efx).max_value(items)



#### Dynamic program definition:


class EF1PartitionDP(SequentialDynamicProgram):

    # The states are of the form (d11, d12, ..., dnn; b11, b12, ..., bnn) where n is the number of agents.
    # where dij := vi(Ai)-vi(Aj).
    # and   bij is the largest value for i of an item allocated to j.

    def __init__(self, valuation_matrix, efx=False):
        num_of_agents = self.num_of_agents = len(valuation_matrix)
        self.thresholds = [
            sum(valuation_matrix[i]) / num_of_agents
            for i in range(num_of_agents)
        ]
        self.valuation_matrix = valuation_matrix
        self.sum_valuation_matrix = sum(map(sum, valuation_matrix))
        self.efx = efx

    def initial_states(self):
        zero_differences = self.num_of_agents * (self.num_of_agents * (0,),)
        # print("zero_differences",zero_differences)
        initial_value_to_remove = math.inf if self.efx else 0
        largest_value_owned_by_others = self.num_of_agents * (
            self.num_of_agents * (initial_value_to_remove,),
        )
        return {(zero_differences, largest_value_owned_by_others)}

    def initial_solution(self):
        empty_bundles = [[] for _ in range(self.num_of_agents)]
        return empty_bundles

    def transition_functions(self):
        return [
            lambda state, input, agent_index=agent_index: (
                _EF1_update_bundle_differences(state[0], agent_index, input),
                _EF1_update_value_owned_by_others(
                    state[1],
                    agent_index,
                    input[-1],
                    self.valuation_matrix,
                    self.efx,
                ),
            )
            for agent_index in range(self.num_of_agents)
        ]

    def construction_functions(self):
        return [
            lambda solution, input, agent_index=agent_index: add_input_to_bin(
                solution, agent_index, input[-1]
            )
            for agent_index in range(self.num_of_agents)
        ]

    def value_function(self):
        return (
            lambda state: (sum(map(sum, state[0])) + self.sum_valuation_matrix)
            / self.num_of_agents
            if self._is_ef1(state[0], state[1])
            else -math.inf
        )

    def _is_ef1(
        self, bundle_differences: list, largest_value_owned_by_others: list
    ) -> bool:
        return all(
            [
                bundle_differences[i][j] + largest_value_owned_by_others[i][j]
                >= 0
                for i in range(self.num_of_agents)
                for j in range(self.num_of_agents)
            ]
        )


def _EF1_update_bundle_differences(bundle_differences, agent_index, item_values):
    """
    >>> _update_bundle_differences( ((0,0),(0,0)), 0, [11,33,0]  )
    ((0, 11), (-33, 0))
    >>> _update_bundle_differences( ((0,0),(0,0)), 1, [11,33,0]  )
    ((0, -11), (33, 0))
    """
    # print("bundle_differences",bundle_differences)
    num_of_agents = len(bundle_differences)
    new_bundle_differences = [list(d) for d in bundle_differences]
    for other_agent_index in range(num_of_agents):
        if other_agent_index == agent_index:
            continue
        new_bundle_differences[agent_index][other_agent_index] += item_values[
            agent_index
        ]
        new_bundle_differences[other_agent_index][agent_index] -= item_values[
            other_agent_index
        ]
    new_bundle_differences = tuple((tuple(d) for d in new_bundle_differences))
    return new_bundle_differences


def _EF1_update_value_owned_by_others(
    largest_value_owned_by_others: list,
    agent_index: int,
    item_index: int,
    valuation_matrix,
    efx=False,
):
    """
    Update the matrix of largest-value-owned-by-others when
    the item #item_index is given to the agent #agent_index.
    >>> _update_value_owned_by_others([[0,0,0],[0,0,0],[0,0,0]], 0, 0, [[55,66,77],[88,99,11],[22,33,44]])
    ((0, 0, 0), (88, 0, 0), (22, 0, 0))
    >>> _update_value_owned_by_others([[0,20,30],[40,0,60],[70,80,0]], 0, 0, [[55,66,77],[88,99,11],[22,33,44]])
    ((0, 20, 30), (88, 0, 60), (70, 80, 0))
    """
    num_of_agents = len(largest_value_owned_by_others)
    new_largest_value_owned_by_others = [
        list(d) for d in largest_value_owned_by_others
    ]
    for other_agent_index in range(num_of_agents):
        if other_agent_index == agent_index:
            continue
        other_agent_value = valuation_matrix[other_agent_index][item_index]
        if efx:
            replace_item = (
                other_agent_value
                < new_largest_value_owned_by_others[other_agent_index][
                    agent_index
                ]
            )
        else:  # ef1
            replace_item = (
                other_agent_value
                > new_largest_value_owned_by_others[other_agent_index][
                    agent_index
                ]
            )
        if replace_item:
            new_largest_value_owned_by_others[other_agent_index][
                agent_index
            ] = other_agent_value
    new_largest_value_owned_by_others = tuple(
        (tuple(d) for d in new_largest_value_owned_by_others)
    )
    return new_largest_value_owned_by_others

#####
## UM within PROP1 Code
####

def utilitarian_prop1_value(valuation_matrix, propx=False):
    """
    Returns the maximum utilitarian value in a PROP1 allocation - does *not* return the partition itself.
    >>> utilitarian_prop1_value([[11,0,11],[33,44,55]])
    132
    >>> utilitarian_prop1_value([[11,0,11],[33,44,55]],propx=True)
    110
    >>> utilitarian_prop1_value([[11,22,33,44],[44,33,22,11]])
    154
    >>> utilitarian_prop1_value([[11,22,33,44],[44,33,22,11]],propx=True)
    154
    >>> utilitarian_prop1_value([[11,0,11,11],[0,11,11,11],[33,33,33,33]])
    132
    >>> utilitarian_prop1_value([[11,0,11,11],[0,11,11,11],[33,33,33,33]],propx=True)
    88
    >>> utilitarian_prop1_value([[11],[22]]) 
    22
    >>> utilitarian_prop1_value([[11],[22]],propx=True)
    22
    """
    items = items_as_value_vectors(valuation_matrix)
    return PROP1PartitionDP(valuation_matrix, propx).max_value(items)


def utilitarian_prop1_allocation(valuation_matrix, propx=False):
    """
    Returns the utilitarian-maximum PROP1 allocation and its utilitarian value.
    >>> dynprog.sequential.logger.setLevel(logging.WARNING)
    >>> utilitarian_prop1_allocation([[11,0,11],[33,44,55]])
    (132, [[], [0, 1, 2]])
    >>> utilitarian_prop1_allocation([[11,0,11],[33,44,55]], propx=True)
    (110, [[0], [1, 2]])
    >>> utilitarian_prop1_allocation([[11,22,33,44],[44,33,22,11]])
    (154, [[2, 3], [0, 1]])
    >>> utilitarian_prop1_allocation([[11,22,33,44],[44,33,22,11]], propx=True)
    (154, [[2, 3], [0, 1]])
    >>> utilitarian_prop1_allocation([[11,0,11,11],[0,11,11,11],[33,33,33,33]])
    (132, [[], [], [0, 1, 2, 3]])
    >>> utilitarian_prop1_allocation([[11,0,11,11],[0,11,11,11],[33,33,33,33]], propx=True)
    (88, [[3], [2], [0, 1]])
    >>> utilitarian_prop1_allocation([[11],[22]]) 
    (22, [[], [0]])
    >>> utilitarian_prop1_allocation([[11],[22]], propx=True)
    (22, [[], [0]])
    >>> utilitarian_prop1_allocation([[37,20,34,12,71,17,55,97,79],[57,5,59,63,92,23,4,36,69],[16,3,41,42,68,47,60,39,17]])
    (574, [[1, 7, 8], [0, 2, 3, 4], [5, 6]])
    >>> utilitarian_prop1_allocation([[37,20,34,12,71,17,55,97,79],[57,5,59,63,92,23,4,36,69],[16,3,41,42,68,47,60,39,17]], propx=True)
    (557, [[7, 8], [0, 2, 3, 4], [1, 5, 6]])
    """
    items = items_as_value_vectors(valuation_matrix)
    (best_state,best_value,best_solution,num_of_states) = PROP1PartitionDP(valuation_matrix, propx).max_value_solution(items)
    if best_value==-math.inf:
        raise ValueError("No proportional allocation")
    return (best_value,best_solution)



#### Dynamic program definition:

class PROP1PartitionDP(SequentialDynamicProgram):

    # The states are of the form  (v1, v2, ..., vn; b1, b2, ..., bn) where n is the number of agents.
    # The "vi" are the value of bundle i to agent i.
    # The "bi" are the largest value for i of an item allocated to others.

    def __init__(self, valuation_matrix, propx=False):
        num_of_agents = self.num_of_agents = len(valuation_matrix)
        self.thresholds = [sum(valuation_matrix[i])/num_of_agents for i in range(num_of_agents)]
        self.valuation_matrix = valuation_matrix
        self.propx = propx

    def initial_states(self):
        zero_values = self.num_of_agents*(0,)
        initial_value_to_remove = math.inf if self.propx else 0
        largest_value_owned_by_others = self.num_of_agents*(initial_value_to_remove,)
        yield (zero_values, largest_value_owned_by_others)

    def initial_solution(self):
        empty_bundles = [ [] for _ in range(self.num_of_agents)]
        return empty_bundles
   
    def transition_functions(self):
        return [
            lambda state, input, agent_index=agent_index: \
                (add_input_to_agent_value(state[0], agent_index, input) , \
                _PROP1_update_value_owned_by_others(state[1], agent_index, input[-1], self.valuation_matrix, self.propx) )
            for agent_index in range(self.num_of_agents)
        ]

    def construction_functions(self):
        return [
            lambda solution,input,agent_index=agent_index: add_input_to_bin(solution, agent_index, input[-1])
            for agent_index in range(self.num_of_agents)
        ]

    def value_function(self):
        return lambda state: sum(state[0]) if self._is_prop1(state[0], state[1]) else -math.inf
    
    def _is_prop1(self, bundle_values:list,largest_value_owned_by_others:list)->bool:
        return all([bundle_values[i] + largest_value_owned_by_others[i] >= self.thresholds[i] for i in range(self.num_of_agents)])





def _PROP1_update_value_owned_by_others(largest_value_owned_by_others:list, agent_index:int, item_index:int, valuation_matrix, propx=False):
    """
    :param input: a list of values: input[i] represents the value of the current item for agent i.
    Adds the given item to agent #agent_index.
    >>> _update_value_owned_by_others([33, 44, 66], 0, 0, [[55,66,77],[88,99,11],[22,33,44]])
    (33, 88, 66)
    """
    logger.info(largest_value_owned_by_others)
    new_largest_value_owned_by_others = list(largest_value_owned_by_others)
    num_of_agents = len(largest_value_owned_by_others)
    for other_agent_index in range(num_of_agents):
        if other_agent_index!=agent_index:
            other_agent_value = valuation_matrix[other_agent_index][item_index]
            if propx:
                should_replace_item = other_agent_value < new_largest_value_owned_by_others[other_agent_index]
            else: # prop1
                should_replace_item = other_agent_value > new_largest_value_owned_by_others[other_agent_index]
            if should_replace_item:
                new_largest_value_owned_by_others[other_agent_index] = other_agent_value
    return tuple(new_largest_value_owned_by_others)



In [16]:
import sys

import numpy as np
valuation_matrix = np.random.randint(0,9, [3,10])
print("valuation_matrix:\n", valuation_matrix)
print("Utilitarian within EF1: ", utilitarian_ef1_value(valuation_matrix))

valuation_matrix = np.random.randint(0,99, [3,7])   # ~ 3000 states
print("Valuation matrix:\n",valuation_matrix)
print("Utilitarian PROP1 value:",utilitarian_prop1_value(valuation_matrix))
print("Utilitarian PROP1 allocation:",utilitarian_prop1_allocation(valuation_matrix))


valuation_matrix:
 [[8 6 0 4 8 5 2 0 4 8]
 [8 5 0 4 8 4 7 7 8 7]
 [2 2 3 0 8 6 0 5 7 1]]
Utilitarian within EF1:  65.0
Valuation matrix:
 [[11 60 41 81 74 96 79]
 [41 30 58 59 64 24 86]
 [55 54 45 13 78 17 92]]
Utilitarian PROP1 value: 520
Utilitarian PROP1 allocation: (520, [[1, 3, 5], [2], [0, 4, 6]])
